## Automation: Weights are all the same?

This here is my process of checking whether or not all the weight columns work similar to each other. I will basically write a Python script that automates what I was doing in Excel.

Steps: 

0. Clean dataset and make it ready 
1. come up with rules for each column: “ex. total spending was Less than 6410 or not” and give 0 or 1 to each entry for the related column.  
2. count frequencies of 1’s (summation of all the rows for that column) 
3. Check: is the frequency number close enough to the frequency in the PUMPF file? 
4. weight column * rule column (”less than 6410”) = weighted frequency column 
5. summation of all the rows of weighted frequency column
6. Check: is the summation of weighted frequency close enough to the weighted frequency in the PUMPF file? 
    1. Yes: Then it’s working similarly 
    2. No: Then it’s different. 
7. Record all progress

### DATA ANALYSIS (short)

In [28]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [29]:
#read data 
og_df = pd.read_csv('wrk_ds_00.csv')
cp_df = og_df.copy() #will work on this 

In [30]:
og_df.head()

,pumfid,province,pop_cent,age_grp,gender,indigen,lang,emp,edu,highestedu,...,wrpg991,wrpg992,wrpg993,wrpg994,wrpg995,wrpg996,wrpg997,wrpg998,wrpg999,wrpg1000
0,100000,11,3,6,1,2,1,2,2,2,...,58.74490,0.00000,91.23504,37.73533,84.68159,43.01466,0.00000,56.83786,103.81114,0.00000
1,100001,59,1,6,2,2,1,2,2,2,...,7136.75133,0.00000,0.00000,5224.34222,9012.71353,2402.81888,4359.56914,2818.29886,0.00000,4805.17223
2,100002,35,1,2,2,2,1,1,2,3,...,0.00000,0.00000,3391.39522,3810.10298,4368.93902,8221.13902,0.00000,0.00000,3462.35492,7660.63229
3,100003,35,1,3,2,2,1,1,2,3,...,6254.81170,6895.04808,4913.14578,5521.61592,0.00000,0.00000,0.00000,15612.81091,0.00000,6483.30541
4,100004,24,1,6,2,2,2,2,2,3,...,1477.61679,0.00000,1217.01163,2722.26852,3028.14230,0.00000,2702.80816,4168.62901,0.00000,0.00000


In [31]:
og_df.columns.get_loc('wtpg')

170

In [32]:
#get some overall information on the dataset 
og_df_sub = og_df.iloc[:, :170]
og_df_sub.head()

,pumfid,province,pop_cent,age_grp,gender,indigen,lang,emp,edu,highestedu,...,dl_speed_more50,life_satisfaction,mentalhealth,visible_minority,disability_status,health,household_type,house_quantile_income,immigrant,household_size
0,100000,11,3,6,1,2,1,2,2,2,...,1,6,2,2,2,2,2,2,2,2
1,100001,59,1,6,2,2,1,2,2,2,...,6,9,2,2,2,2,2,4,2,2
2,100002,35,1,2,2,2,1,1,2,3,...,6,9,3,2,2,3,1,3,2,3
3,100003,35,1,3,2,2,1,1,2,3,...,6,9,2,2,2,1,2,2,2,3
4,100004,24,1,6,2,2,2,2,2,3,...,6,11,1,2,2,3,2,4,2,2


In [33]:
df_noWeights = og_df_sub.copy()

In [34]:
only_weights = og_df.iloc[:, 170:]
only_weights.head()

,wtpg,wrpg1,wrpg2,wrpg3,wrpg4,wrpg5,wrpg6,wrpg7,wrpg8,wrpg9,...,wrpg991,wrpg992,wrpg993,wrpg994,wrpg995,wrpg996,wrpg997,wrpg998,wrpg999,wrpg1000
0,45.17965,149.11093,0.00000,149.63066,0.00000,84.74563,0.00000,182.72995,50.43055,153.19554,...,58.74490,0.00000,91.23504,37.73533,84.68159,43.01466,0.00000,56.83786,103.81114,0.00000
1,2524.87054,4341.00760,7856.51990,4982.54580,2876.07653,2990.92407,9237.73885,2315.91625,2395.59320,5015.74080,...,7136.75133,0.00000,0.00000,5224.34222,9012.71353,2402.81888,4359.56914,2818.29886,0.00000,4805.17223
2,3720.74855,0.00000,3470.18526,10686.85924,0.00000,7389.08920,3743.67150,3538.84146,4176.14775,4105.81575,...,0.00000,0.00000,3391.39522,3810.10298,4368.93902,8221.13902,0.00000,0.00000,3462.35492,7660.63229
3,5802.87880,6201.46077,0.00000,0.00000,5982.75992,6253.22554,12333.67688,5744.61028,10343.24866,5259.79450,...,6254.81170,6895.04808,4913.14578,5521.61592,0.00000,0.00000,0.00000,15612.81091,0.00000,6483.30541
4,1449.85202,1522.92124,1711.91466,1448.57857,0.00000,0.00000,2833.25017,1547.04205,1316.13353,2450.45243,...,1477.61679,0.00000,1217.01163,2722.26852,3028.14230,0.00000,2702.80816,4168.62901,0.00000,0.00000


In [35]:
df_noWeights.iloc[:,:50].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17409 entries, 0 to 17408
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   pumfid                17409 non-null  int64
 1   province              17409 non-null  int64
 2   pop_cent              17409 non-null  int64
 3   age_grp               17409 non-null  int64
 4   gender                17409 non-null  int64
 5   indigen               17409 non-null  int64
 6   lang                  17409 non-null  int64
 7   emp                   17409 non-null  int64
 8   edu                   17409 non-null  int64
 9   highestedu            17409 non-null  int64
 10  access_internet       17409 non-null  int64
 11  home_use              17409 non-null  int64
 12  noUse_need            17409 non-null  int64
 13  noUse_difficult       17409 non-null  int64
 14  noUse_security        17409 non-null  int64
 15  access_dataplan       17409 non-null  int64
 16  acce

In [36]:
df_noWeights.iloc[:,50:100].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17409 entries, 0 to 17408
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   info_news               17409 non-null  int64
 1   info_gps                17409 non-null  int64
 2   info_health             17409 non-null  int64
 3   info_events             17409 non-null  int64
 4   info_services           17409 non-null  int64
 5   listen_music            17409 non-null  int64
 6   listen_podcast          17409 non-null  int64
 7   listen_news             17409 non-null  int64
 8   watch_streaming         17409 non-null  int64
 9   watch_video             17409 non-null  int64
 10  watch_sports            17409 non-null  int64
 11  watch_Esports           17409 non-null  int64
 12  watch_videoGame         17409 non-null  int64
 13  watch_liveStream        17409 non-null  int64
 14  time_spent_watch        17409 non-null  int64
 15  act_videoGame      

In [37]:
df_noWeights.iloc[:,100:150].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17409 entries, 0 to 17408
Data columns (total 50 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   spend_online_dig_books       17409 non-null  int64
 1   spend_online_dig_podcast     17409 non-null  int64
 2   spend_online_dig_news        17409 non-null  int64
 3   spend_online_dig_giftcards   17409 non-null  int64
 4   spend_online_dig_gambling    17409 non-null  int64
 5   spend_online_dig_gaming      17409 non-null  int64
 6   spend_online_dig_storage     17409 non-null  int64
 7   spend_online_dig_courses     17409 non-null  int64
 8   spend_online_dig_software    17409 non-null  int64
 9   spend_online_dig_other       17409 non-null  int64
 10  total_online_spending        17409 non-null  int64
 11  spend_online_phy_total       17409 non-null  int64
 12  purchase_phy_goods           17409 non-null  int64
 13  sub_mealKit                  17409 non-null  i

In [38]:
df_noWeights.iloc[:,150:200].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17409 entries, 0 to 17408
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   cyber_incident_payment     17409 non-null  int64
 1   cyb_inc                    17409 non-null  int64
 2   cyberAttack_financialLoss  17409 non-null  int64
 3   change_card_cyberAttack    17409 non-null  int64
 4   workType                   17409 non-null  int64
 5   WFH_hours                  17409 non-null  int64
 6   WFH_loc_home               17409 non-null  int64
 7   WFH_loc_officeSpace        17409 non-null  int64
 8   online_earnings            17409 non-null  int64
 9   dl_speed_less50            17409 non-null  int64
 10  dl_speed_more50            17409 non-null  int64
 11  life_satisfaction          17409 non-null  int64
 12  mentalhealth               17409 non-null  int64
 13  visible_minority           17409 non-null  int64
 14  disability_status     

In [39]:
df_noWeights.describe()

,pumfid,province,pop_cent,age_grp,gender,indigen,lang,emp,edu,highestedu,...,dl_speed_more50,life_satisfaction,mentalhealth,visible_minority,disability_status,health,household_type,house_quantile_income,immigrant,household_size
count,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,...,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000,17409.000000
mean,108704.000000,31.470561,1.402838,4.406169,1.533230,2.320179,1.937791,1.828594,2.286116,2.315871,...,5.532081,13.629789,2.734160,2.231604,2.276236,2.757252,2.176805,2.788156,1.907404,2.282038
std,5025.689754,15.801214,0.610865,1.549597,0.498909,1.541426,1.832290,1.629246,1.543522,1.751280,...,1.279575,21.152269,1.802006,1.614541,1.611628,1.756032,1.000913,1.397352,0.350185,1.170672
min,100000.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,104352.000000,13.000000,1.000000,3.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,...,6.000000,8.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
50%,108704.000000,35.000000,1.000000,5.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,...,6.000000,9.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,2.000000
75%,113056.000000,46.000000,2.000000,6.000000,2.000000,2.000000,3.000000,2.000000,2.000000,3.000000,...,6.000000,10.000000,3.000000,2.000000,2.000000,3.000000,3.000000,4.000000,2.000000,3.000000
max,117408.000000,59.000000,3.000000,6.000000,2.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,99.000000,9.000000,9.000000,9.000000,9.000000,9.000000,5.000000,9.000000,5.000000


As is shown above, everything is being looked at as an integer. Which is wrong. These are mostly all categorical. 

### DATA Explanations 

In here, I will add information about the dataset in the form of code. 

In [ ]:
province_codes = {
    'NFL': 10,
    'PRNEDW': 11,
    'NScotia': 12,
    'NBrun': 13,
    'Qbc': 24,
    'ON': 35,
    'Mntb': 46,
    'Sask': 47,
    'Albt': 48,
    'BC': 59,
}

population_centre_codes = {
    'urban': 1,
    'rural': 2,
    'PRNEDW island': 3
}

age_codes = {
    '15-24': 1,
    '25-34': 2,
    '35-44': 3,
    '45-54': 4,
    '55-64': 5,
    '65+': 6
}

gender_codes = {
    'male': 1,
    'female': 2
}

aboriginal_codes = {
    'aboriginal': 1,
    'nonabo': 2
}

language_codes = {
    'ENG': 1,
    'FRN': 2,
    'ENG and FRN': 3,
    'NO ENG NO FRN': 4
}

employement_codes = {
    'employed': 1,
    'unemployed': 2
}

student_codes = {
    'student': 1, 
    'not student': 2
}

education_codes = {
    'HS or less': 1,
    'Some post-secondary': 2,
    'Uni degree': 3
}

mobile_data_size_codes = {
    'less than 1GB': 1,
    '1 to 6 GB': 2,
    '6 to 10 GB': 3,
    '10 to 20 GB': 4,
    '20 to 30 GB': 5,
    '30GB+': 6
}

last_use_internet_codes = {
    '3 months to a year ago': 1,
    'more than 1 year ago': 2,
    'never': 3
}

frequency_smartphone_use_codes = {
    'every 5 mins': 1,
    'every 15 mins': 2,
    'every 30 mins': 3,
    'every hour': 4,
    'a few times': 5,
    'only once or less': 6
}

friend_rel_codes = {
    'completely dissatisfied': 1,
    'somewhat dissatisfied': 2,
    'neutral': 3,
    'somewhat satisfied': 4,
    'completely satisfied': 5
}

family_rel_codes = {
    'completely dissatisfied': 1,
    'somewhat dissatisfied': 2,
    'neutral': 3,
    'somewhat satisfied': 4,
    'completely satisfied': 5
}

useNet_comm_friends_codes = {
    'everyday': 1,
    'few times a week': 2,
    'once a week': 3,
    '2-3 times a month': 4,
    'once a month': 5,
    '0 the past month': 6,
    'not applicable': 7
}

useNet_plan_withFriends_codes = {
    'everyday': 1,
    'few times a week': 2,
    'once a week': 3,
    '2-3 times a month': 4,
    'once a month': 5,
    '0 the past month': 6,
    'not applicable': 7
}

useNet_comm_family_codes = {
    'everyday': 1,
    'few times a week': 2,
    'once a week': 3,
    '2-3 times a month': 4,
    'once a month': 5,
    '0 the past month': 6,
    'not applicable': 7
}

useNet_plan_withFamily_codes = {
    'everyday': 1,
    'few times a week': 2,
    'once a week': 3,
    '2-3 times a month': 4,
    'once a month': 5,
    '0 the past month': 6,
    'not applicable': 7
}

purchase_physicalGoods_frq_codes = {
    'weekly': 1,
    'few times a month': 2,
    'once a month': 3,
    'few times a year': 4
}

loyalty_subscription_codes = {
    'less than 200': 1,
    '200 to 500': 2,
    '500 to 1000': 3,
    '1000 to 5000': 4,
    '5000 or more': 5
}

onlineShop_security_concern_codes = {
    'extremely concerned': 1,
    'very concerned': 2,
    'somewhat concerned': 3,
    'a little bit concerned': 4,
    'not concerned at all': 5
}

socialMedia_security_concern_codes = {
    'extremely concerned': 1,
    'very concerned': 2,
    'somewhat concerned': 3,
    'a little bit concerned': 4,
    'not concerned at all': 5
}

workPlace_type_codes = {
    'fixed loc outside of home': 1,
    'non fixed loc outside of home': 2,
    'work at home': 3
}

online_earnings_codes = {
    '0': 1,
    '1 to 200': 2,
    '200 to 1000': 3,
    '1000 to 2000': 4,
    '2000 or more': 5
}

downloadSpeeds_lessthan50_codes = {
    'less than 50Mbps': 1,
    'More than 50Mbps': 2
}

downloadSpeeds_morethan50_codes = {
    'less than 5 Mbps': 1,
    '5 to 20': 2,
    '20 to 50': 3
}

# some covid related data points here... 

life_satisfaction_codes = {
    'extremely dissatisfied': 1,
    'very dissatisfied': 2,
    'dissatisfied': 3,
    'somewhat dissatisfied': 4,
    'neutral': 5,
    'somewhat satisfied': 6,
    'satisfied': 7,
    'very satisfied': 8,
    'extremely satisfied': 9,
    'Incredibly satisfied': 10
}

mentalHealth_codes = {
    'Excellent': 1,
    'Very good': 2,
    'Good': 3,
    'Fair': 4,
    'Poor': 5
}

physicalHealth_codes = {
    'Excellent': 1,
    'Very good': 2,
    'Good': 3,
    'Fair': 4,
    'Poor': 5
}

household_type_codes = {
    'kids under 18': 1,
    'no kids under 18': 2,
    'single person': 3,
    'other': 4
}

household_income_quantile_codes = {
    'Q1: lOE to 44,119': 1,
    'Q2: 44,120 to 75321': 2,
    'Q3: 75322 to 109431': 3,
    'Q4: 109432 to 162799': 4,
    'Q5: GOE to 162800': 5
}

household_size_codes = { #this one somehow aligns tho
    '1 person': 1,
    '2 people': 2,
    '3 people': 3,
    '4 people': 4,
    '5 or more people': 5
}

Continous Variables: 

EC_G010A, music streaming subs dollar amounts, 0 - 450 
EC_G010B, video streaming subs dollar amounts, 0 - 660 
EC_G010C, Ebooks subs dollar amounts, 0 - 360 
EC_G010D, Podcasts subs dollar amounts, 0 - 300
EC_G010E, Newspapers subs dollar amounts, 0 - 540 
EC_G010F, Giftcards purchased dollar amounts, 0 - 600
G, Gambling dollar amounts, 0 - 1200
H, Gaming dollar amount, 0 - 600
I, Data Storage dollar amount, 0 - 180
J, Courses dollar amount, 0 - 1500
K, Applications and software dollar amounts, 0 - 450 

EC_G010X, total spending online (digital goods), $ 0 - 6410 

==================

EC_G020A, physical goods total $ amount: $ 0 - 6000

==================

EC_G050B, P2P accommodation services total amount spent: $ 0 - 3900

EC_G060A, Online purchases services total amount spent: $ 0 - 3000

SUM_GTOT, Total spending online: $ 0 - 18310 




### STEP1. Information

This is a summary information from the pdf file in a List based on the pattern where every entry is a dictionary with the following information:  

```python

column_name = {
    'name': col_name,
    'position': 1, #integer value 
    'category': cat_type #type of category 
}

```


In [3]:
summary_info =  {
        'name': ,
        'position':,
        'category': 
    }

lookUp_info = #input all the column informations here. 

#task: fill this out 

SyntaxError: invalid syntax (<ipython-input-3-68dbabc259dd>, line 2)

### STEP2. GET INFORMATION 

Here, I'll write a function to gather information by looking up the position and fetching the weight column. 

In [8]:
def get_info(col_name):
    """
    Looks into the LookUp list: 
        - finds the position of column, 
        - finds the categorization method for column 
        - returns a dictionary with the following info:
            * dataframe with column itself, weight column corresponding to it 
            * categorization method 
            * position number 
    """
    
    for i in range(len(lookUp_info)):
        if lookUp_info[i]['name'] == col_name: 
            pos = lookUp_info[i]['position']
            cat = lookUp_info[i]['category']
        else:
            continue
    
    cols_ = [col_name, 'wts_{}'.format(col_name)]
    s1 = pd.Series(cp_df[col_name])
    s2 = pd.Series(cp_df['wrpg{}'.format(pos)])
    df_ = pd.concat([s1, s2], axis = 1, columns = cols_)
    
    ret_dict = {
        'data': df_,
        'pos': pos,
        'cat': cat
    }
    return ret_dict 
    

### STEP3. CATEGORIZATIONS. 

This step gives information about how to give 0's and 1's to the data based on the question that was asked (how to categorize the answers). Some of the columns (questions) have more than 2 choices of answers, this means I will have to add a column for each answer and do 0's and 1's. For example, with age it was 15-25 (01), 25-34 (02), ..., 65 and older (06), etc. To save time, I'll just pick 1 of these and check. So, with age it will be "15-25"--- it's either 15-25 or it isn't. Then I'll check all the 15-24's and make my decision. 

In [9]:
def replace_with_binary(l, value_to_check, f):
    """
    This is just to avoid repeatedly writing a loop and if statement
    """
    f = [] 
    for i in len(range(l)):
        if li[i] == value_to_check: 
            f.insert(i, 1)
        else:
            f.insert(i, 0)
    
    return f

In [ ]:
def categorize(col_name, categ):
    """
    Knowing the category, make a decision on how to fill up a frequency column. 
    """
#     freq = [] 
    
    li = list(cp_df[col_name])
    
    def replace
    
    match categ:
        case "ON":
            #ontario gets 1, everywhere else gets 0 
#             freq = replace_with_binary(li, "35", freq) #MIGHT be an issue: passing freq, replacing it. 
            freq = replace_with_binary(li, '35')
        case "Pop":
            #population centre, urban centres get 1, everywhere else gets 0 
            freq = replace_with_binary(li, "01")
            
        case "age":
            #age group, 15-24 gets 1, everything else gets 0 
            freq = replace_with_binary(li, '01')
            
        case "sex":
            #gender, Female gets 1, all else 0 
            freq = replace_with_binary(li, '1')
            
        case "aboriginal":
            #aboriginal gets 1, all else 0 
            freq = replace_with_binary(li, '1')
            
        case "lang":
            #English 1
            freq = replace_with_binary(li, '1')
            
        case "employement":
            #employed 1 
            freq = replace_with_binary(li, '1')
        
        case "student":
            #currently a student: 1
            freq = replace_with_binary(li, '1')
            
        case "education":
            #highest education is Highschool: 1
            freq = replace_with_binary(li, '1')
            
        case "internet_access":
            #has internet access: 1 
            freq = replace_with_binary(li, '1')
            
        case "home_use":
            #has access to internet at home: 1 
            freq = replace_with_binary(li, '1')
            
        case "noaccess_r1":
            #cost of internet too high, so no access: YES, 1
            freq = replace_with_binary(li, '1')
        
        case "noaccess_r2":
            #cost of equipment too high, so no internet access: Yes, 1
            freq = replace_with_binary(li, '1')
            
        case "noaccess_r3":
            #no need for internet, so no access: Yes, 1
            freq = replace_with_binary(li, '1')
        
        case "noaccess_r4":
            #too hard or disabled persons, yes 1 
            freq = replace_with_binary(li, '1')
        
        case "noaccess_r5":
            #have access somewhere else, yes 1 
            freq = replace_with_binary(li, '1')
            
        case "noaccess_r6":
            #service not available or it's not good, yes: 1 
            freq = replace_with_binary(li, '1')
            
        case "noaccess_r7":
            #security issues or concerns, yes : 1 
            freq = replace_with_binary(li, '1')
        
        case "mobileData":
            #use mobile data, yes 1 
            freq = replace_with_binary(li, '1')
            
        case "mobileDataSize":
            #less than 1 GB/month: 1 
            freq = replace_with_binary(li, '01')
            
        case ""